In [3]:
# STEP 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# STEP 2: Combine CSV files
csv_files = [
    '/content/financial_data_batch_0.csv',
    '/content/financial_data_batch_1.csv',
    '/content/financial_data_batch_2.csv',
    '/content/financial_data_batch_3.csv',
    '/content/financial_data_batch_4.csv'
]

combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
combined_df.to_csv('combined_output.csv', index=False)
print(" Combined CSVs into 'combined_output.csv'")

# STEP 3: Filter required tags and pivot the data
tags = [
    "RevenueFromContractWithCustomerExcludingAssessedTax", "GrossProfit", "NetIncomeLoss",
    "ResearchAndDevelopmentExpense", "Assets", "Liabilities", "StockholdersEquity",
    "OperatingIncomeLoss", "AssetsCurrent", "LiabilitiesCurrent"
]

df = pd.read_csv("combined_output.csv")
df_filtered = df[df['tag'].isin(tags)]

pivot = df_filtered.pivot_table(
    index=['name', 'cik'],
    columns='tag',
    values='value',
    aggfunc='first'
).reset_index()

# STEP 4: Calculate financial ratios
pivot['NetMargin'] = pivot['NetIncomeLoss'] / pivot['RevenueFromContractWithCustomerExcludingAssessedTax']
pivot['GrossMargin'] = pivot['GrossProfit'] / pivot['RevenueFromContractWithCustomerExcludingAssessedTax']
pivot['R&D_to_Revenue'] = pivot['ResearchAndDevelopmentExpense'] / pivot['RevenueFromContractWithCustomerExcludingAssessedTax']
pivot['ROA'] = pivot['NetIncomeLoss'] / pivot['Assets']
pivot['ROE'] = pivot['NetIncomeLoss'] / pivot['StockholdersEquity']
pivot['DebtToEquity'] = pivot['Liabilities'] / pivot['StockholdersEquity']
pivot['CurrentRatio'] = pivot['AssetsCurrent'] / pivot['LiabilitiesCurrent']
pivot['ROS'] = pivot['OperatingIncomeLoss'] / pivot['RevenueFromContractWithCustomerExcludingAssessedTax']

# STEP 5: Clean missing and infinite values
pivot.replace([np.inf, -np.inf], np.nan, inplace=True)
pivot = pivot.dropna(subset=[
    'RevenueFromContractWithCustomerExcludingAssessedTax',
    'NetIncomeLoss', 'Assets', 'Liabilities', 'StockholdersEquity'
])

# STEP 6: Export cleaned and calculated data
pivot.to_csv("financial_ratios_cleaned.csv", index=False)
print("✅ Exported final cleaned and enriched data to 'financial_ratios_cleaned.csv'")
